In [2]:
#data parsing 
import json
import pandas as pd

In [3]:
def parse_data(json_file):
    # Initialize an empty list to hold the parsed data
    parsed_data = []

    # Open the file and read line by line
    with open(json_file, 'r') as f:
        for line in f:
            if line.strip():  # Skip empty lines
                try:
                    # Load the JSON data from the current line
                    json_data = json.loads(line)

                    # Iterate through the transcripts
                    for transcript_id, positions in json_data.items():
                        # Iterate through each position in the transcript
                        for pos, nucleotides in positions.items():
                            # Convert position to integer
                            pos = int(pos)  # Ensure 'transcript_position' is an integer
                            
                            # Iterate through the nucleotide combinations
                            for nucleotide_seq, reads in nucleotides.items():
                                # Iterate through each read and extract features
                                for read in reads:
                                    # Create a dictionary to store features for this read
                                    features = {
                                        'transcript_id': transcript_id,
                                        'transcript_position': pos,
                                        'nucleotide_sequence': nucleotide_seq,
                                        'dwelling_time': read[0],
                                        'std_dev': read[1],
                                        'mean_signal': read[2],
                                        'dwelling_time_flank1': read[3],
                                        'std_dev_flank1': read[4],
                                        'mean_signal_flank1': read[5],
                                        'dwelling_time_flank2': read[6],
                                        'std_dev_flank2': read[7],
                                        'mean_signal_flank2': read[8],
                                    }
                                    # Append the features dictionary to the list
                                    parsed_data.append(features)

                except json.JSONDecodeError as e:
                    print(f"Error parsing line: {e}")

    # Convert the list of parsed data into a DataFrame
    df = pd.DataFrame(parsed_data)
    return df  # Return the DataFrame

# Main code block
if __name__ == "__main__":
    jsonfile = "C:/uni/y4s1/DSA4262/grpproj/dataset0.json"  # Replace with your actual JSON file path
    parsed_df = parse_data(jsonfile)  # Get the DataFrame from parse_data
    print(parsed_df.head())


     transcript_id  transcript_position nucleotide_sequence  dwelling_time  \
0  ENST00000000233                  244             AAGACCA        0.00299   
1  ENST00000000233                  244             AAGACCA        0.00631   
2  ENST00000000233                  244             AAGACCA        0.00465   
3  ENST00000000233                  244             AAGACCA        0.00398   
4  ENST00000000233                  244             AAGACCA        0.00664   

   std_dev  mean_signal  dwelling_time_flank1  std_dev_flank1  \
0     2.06        125.0               0.01770           10.40   
1     2.53        125.0               0.00844            4.67   
2     3.92        109.0               0.01360           12.00   
3     2.06        125.0               0.00830            5.01   
4     2.92        120.0               0.00266            3.94   

   mean_signal_flank1  dwelling_time_flank2  std_dev_flank2  \
0               122.0               0.00930           10.90   
1             

In [4]:
# Data Preparation - Original Dataset

# Load the m6A labels from the original data.info.labelled file
original_labels_df = pd.read_csv("C:/uni/y4s1/DSA4262/grpproj/data.info.labelled", header=None, names=['gene_id', 'transcript_id', 'transcript_position', 'label'])

# Ensure 'transcript_position' in original_labels_df is an integer
original_labels_df['transcript_position'] = pd.to_numeric(original_labels_df['transcript_position'], errors='coerce')

# Merge features DataFrame (parsed_df) with original labels DataFrame
original_merged_df = pd.merge(parsed_df, original_labels_df, on=['transcript_id', 'transcript_position'], how='left')

# Step 1: Define features (X) and target variable (y) for the original dataset
y_original = original_merged_df['label']  # Binary label for m6A modifications (original)
X_original = original_merged_df.drop(columns=['transcript_id', 'transcript_position', 'nucleotide_sequence', 'label', 'gene_id'])

# Step 2: Convert numeric columns and drop non-numeric ones for the original dataset
X_original = X_original.apply(pd.to_numeric, errors='coerce')  # Convert to numeric and set invalid parsing to NaN
X_original = X_original.dropna()  # Drop rows with NaN values

# Optionally, print shapes to verify
print(f"Original Dataset - Features shape: {X_original.shape}, Labels shape: {y_original.shape}")


Original Dataset - Features shape: (11027106, 9), Labels shape: (11027106,)


In [5]:
# Data Preparation - Balanced Dataset (undersampled)

# Load the m6A labels from the balanced_labels.csv file
underbalanced_labels_df = pd.read_csv("C:/uni/y4s1/DSA4262/grpproj/underbalanced_labels.csv")

# Ensure 'transcript_position' in balanced_labels_df is an integer
underbalanced_labels_df['transcript_position'] = pd.to_numeric(underbalanced_labels_df['transcript_position'], errors='coerce')

# Merge features DataFrame (parsed_df) with balanced labels DataFrame
underbalanced_merged_df = pd.merge(parsed_df, underbalanced_labels_df, on=['transcript_id', 'transcript_position'], how='left')
underbalanced_merged_df = underbalanced_merged_df.dropna(subset=['label'])

# Step 1: Define features (X) and target variable (y) for the balanced dataset
y_underbalanced = underbalanced_merged_df['label']  # Binary label for m6A modifications (balanced)
X_underbalanced = underbalanced_merged_df.drop(columns=['transcript_id', 'transcript_position', 'nucleotide_sequence', 'label', 'gene_id'])

# Step 2: Convert numeric columns and drop non-numeric ones for the balanced dataset
X_underbalanced = X_underbalanced.apply(pd.to_numeric, errors='coerce')  # Convert to numeric and set invalid parsing to NaN
X_underbalanced = X_underbalanced.dropna()  # Drop rows with NaN values

# Optionally, print shapes to verify
print(f"Undersampled Balanced Dataset - Features shape: {X_underbalanced.shape}, Labels shape: {y_underbalanced.shape}")


Undersampled Balanced Dataset - Features shape: (1230170, 9), Labels shape: (1230170,)


Model Training with original labelled data

In [6]:
# Import necessary libraries for evaluation metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
import numpy as np

In [7]:
# Assuming X_original contains the features and y_original contains the labels
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_original, y_original, test_size=0.3, random_state=42)

# Initialize the Random Forest classifier
rf_model_original = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model_original.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model_original.predict(X_test)

print("Unique values in y_test:", set(y_test))
print("Unique values in y_pred:", set(y_pred))

Unique values in y_test: {'0', '1'}
Unique values in y_pred: {'0', '1'}


In [9]:
# save model
import pickle
with open('rf_model_original.pkl','wb') as f:
    pickle.dump(rf_model_original,f)

In [10]:
#  load model
with open('rf_model_original.pkl', 'rb') as f:
    rf_model_original = pickle.load(f)

In [13]:
# Ensure predictions and test labels are integers
y_test = np.array(y_test)
y_test = y_test.astype(int)
y_pred = y_pred.astype(int)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary', zero_division=0)  # Handle cases with no positive predictions
recall = recall_score(y_test, y_pred, average='binary', zero_division=0)
f1 = f1_score(y_test, y_pred, average='binary', zero_division=0)

# Binarize the labels for ROC AUC calculation
lb = LabelBinarizer()
y_test_binarized = lb.fit_transform(y_test)
y_pred_proba = rf_model_original.predict_proba(X_test)[:, 1]  # Probabilities for the positive class
auc_roc = roc_auc_score(y_test_binarized, y_pred_proba)

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC-ROC: {auc_roc:.4f}")

Accuracy: 0.9552
Precision: 0.5606
Recall: 0.0427
F1 Score: 0.0794
AUC-ROC: 0.7828


Model Training with balanced labelled data

In [14]:
# Split the dataset into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X_underbalanced, y_underbalanced, test_size=0.3, random_state=42)

# Initialize the Random Forest classifier
rf_model_under = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model_under.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model_under.predict(X_test)

In [15]:
# save model
import pickle
with open('rf_model_under.pkl','wb') as f:
    pickle.dump(rf_model_under,f)

In [16]:
#  load model
with open('rf_model_under.pkl', 'rb') as f:
    rf_model_under = pickle.load(f)

In [17]:
# Ensure predictions and test labels are integers
y_test = np.array(y_test)
y_test = y_test.astype(int)
y_pred = y_pred.astype(int)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary', zero_division=0)  # Handle cases with no positive predictions
recall = recall_score(y_test, y_pred, average='binary', zero_division=0)
f1 = f1_score(y_test, y_pred, average='binary', zero_division=0)

# Binarize the labels for ROC AUC calculation
lb = LabelBinarizer()
y_test_binarized = lb.fit_transform(y_test)
y_pred_proba = rf_model_under.predict_proba(X_test)[:, 1]  # Probabilities for the positive class
auc_roc = roc_auc_score(y_test_binarized, y_pred_proba)

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC-ROC: {auc_roc:.4f}")

Accuracy: 0.7294
Precision: 0.6913
Recall: 0.6035
F1 Score: 0.6444
AUC-ROC: 0.7993


Accuracy:
Original Dataset (0.9552): A high accuracy indicates that the model correctly predicts the majority of instances. However, this high accuracy can be misleading in genomics, particularly with imbalanced datasets where m6A modifications (positive class) may be rare compared to non-modified sites (negative class).
Balanced Dataset (0.7294): The lower accuracy reflects a more realistic assessment, as it considers the performance of the model on both modified and unmodified sites.

Precision:
Original Dataset (0.5606): Precision indicates the proportion of true m6A modifications among all predicted modifications. A lower precision suggests that many predicted m6A sites may be false positives, leading to unreliable results in downstream applications.
Balanced Dataset (0.6913): A higher precision means that the model is better at correctly identifying true m6A sites. This is critical in genomics, where a high rate of false positives can mislead research conclusions and experimental designs.

Recall:
Original Dataset (0.0427): The very low recall indicates that the original model fails to identify most of the actual m6A modifications. In the context of genomic research, missing m6A sites can result in significant gaps in understanding gene regulation.
Balanced Dataset (0.6035): A significantly higher recall shows that the balanced model is much more effective in capturing m6A sites. This ability is crucial for genomic studies, as missing m6A sites could lead to incomplete analyses of gene expression and regulatory networks.

F1 Score:
Original Dataset (0.0794): The F1 score, which balances precision and recall, is low, suggesting that the model's overall effectiveness is poor in making reliable m6A predictions.
Balanced Dataset (0.6444): A much higher F1 score indicates that the balanced model strikes a better balance between precision and recall, enhancing its utility in genomics research.

AUC-ROC:
Original Dataset (0.7828): While the AUC-ROC indicates some ability to distinguish between m6A and non-m6A sites, it doesn't reflect the model's reliability in practice due to the imbalanced nature of the data.
Balanced Dataset (0.7993): The slight improvement in AUC-ROC reinforces that the balanced model has a better capacity to differentiate between the two classes, which is essential for accurate biological interpretations.

Conclusion 
The undersampled balanced dataset provides a more reliable model for m6A predictions. It effectively captures both true m6A sites and reduces false positives, making it a valuable tool for researchers studying gene regulation and m6A modification patterns. While it shows high accuracy, its poor performance in recall and precision highlights significant shortcomings in capturing the complexities of m6A modifications. This model may mislead researchers by suggesting high confidence in predictions that may not hold true in practical applications. Overall, the balanced model's superior performance across all relevant metrics enhances its applicability in genomics research, enabling better insights into m6A modifications and their biological implications.

Now that we concluded that the balanced data set...

Hyperparameter tuning (Hyperopt)

In [36]:
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer

In [ ]:
# Split the dataset into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)

# Define the objective function for Hyperopt
def objective(params):
    rf_model = RandomForestClassifier(**params, random_state=42)
    
    # Use cross-validation to evaluate model performance
    cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='roc_auc')  # Using ROC AUC for optimization
    mean_auc = np.mean(cv_scores)
    
    # Return the negative mean AUC (because Hyperopt minimizes the objective function)
    return {'loss': -mean_auc, 'status': STATUS_OK}

# Define the parameter space for Hyperopt
space = {
    'n_estimators': hp.choice('n_estimators', range(50, 300)),  # Number of trees in the forest
    'max_depth': hp.choice('max_depth', range(10, 50)),         # Maximum depth of each tree
    'min_samples_split': hp.choice('min_samples_split', range(2, 10)),  # Minimum samples to split a node
    'min_samples_leaf': hp.choice('min_samples_leaf', range(1, 10)),    # Minimum samples at a leaf node
    'max_features': hp.choice('max_features', ['auto', 'sqrt', 'log2']), # Number of features to consider
    'bootstrap': hp.choice('bootstrap', [True, False])          # Whether to bootstrap samples
}

# Initialize Trials object to keep track of Hyperopt's progress
trials = Trials()

# Run the Hyperopt optimization
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

# Print best hyperparameters found
print(f"Best hyperparameters: {best_params}")